[Теория](https://programforyou.ru/poleznoe/convolutional-network-from-scratch-part-zero-introduction)

1. С помощью библиотеки numpy реализовать модель с прямым проходом, состоящую из 3 полносвязных слоёв с функциями потерь: ReLU, tanh, Softmax. Длины векторов на входе 256, на выходе 4, промежуточные: 64 и 16.

*Веса задаются случайно в диапазне от -1 до 1, сдвиги не участвуют*

In [14]:
import numpy as np
from abc import abstractmethod


# activation functions:
def softmax(X):
    denominator_sum = np.sum(np.exp(X))
    return np.exp(X) / denominator_sum


def ReLU(X):
    return np.maximum(0, X)


def tanh(X):
    return np.tanh(X)

In [15]:
class Layer:
    @abstractmethod
    def forward(self, X):
        pass

In [16]:
class FCLayer(Layer):
    def __init__(self, size_W, activation_function):
        self.W = np.random.uniform(-1, 1, size_W)
        self.activation_function = activation_function

    def forward(self, X):
        # Y = f(XW)
        Y = X.dot(self.W)
        return self.activation_function(Y)

In [17]:
class Model:
    @abstractmethod
    def __init__(self):
        self.layers = []

    def __call__(self, data):
        res = data
        for layer in self.layers:
            res = layer.forward(res)
        return res

In [18]:
class Model_1(Model):
    def __init__(self):
        layer_1 = FCLayer((256, 64), ReLU)
        layer_2 = FCLayer((64, 16), tanh)
        layer_3 = FCLayer((16, 4), softmax)
        self.layers = [layer_1, layer_2, layer_3]

In [19]:
np.random.seed(19193)
X = np.random.randint(0, 255, size=256)
model_1 = Model_1()
model_1(X)

array([0.04262191, 0.76603391, 0.00249729, 0.18884689])

2. Реализовать модель с прямым проходом, состоящую из 2 свёрток с функциями активации ReLU и 2 функций MaxPool. Первый слой переводит из 3 каналов в 8, второй из 8 слоёв в 16. На вход подаётся изображение размера 19х19. (19х19x3 -> 18x18x8 -> 9x9x8 -> 8x8x16 -> 4x4x16)

In [20]:
np.random.seed(19193)

In [21]:
class ConvolutionLayer(Layer):
    def __init__(self, channels_in, channels_out, kernel_size=3, step=1, activation_function=ReLU):
        self.channels_out = channels_out
        self.activation_function = activation_function
        self.step = step

        # набор 3D-фильтров
        self.filters = []
        for _ in range(channels_out):
            self.filters.append(np.random.random((kernel_size, kernel_size, channels_in)))
        self.filters = np.array(self.filters)

    def __convolution_3d(self, matrix, filter):
        if matrix.shape[2] != filter.shape[2]:
            raise ValueError("The number of channels is not the same!")

        res = np.zeros((matrix.shape[0] - 1, matrix.shape[1] - 1, matrix.shape[2]))
        # 2D свертка по всем каналам
        for channel in range(matrix.shape[2]):
            kernel = filter[:, :, channel]
            matrix_2d = matrix[:, :, channel]
            for y in range(0, matrix.shape[0] - kernel.shape[0] + 1, self.step):
                for x in range(0, matrix.shape[1] - kernel.shape[1] + 1, self.step):
                    res[y][x][channel] = np.sum(matrix_2d[ y:(y + kernel.shape[0]), x:(x + kernel.shape[1]) ] * kernel)
        # сложение слоев
        return np.sum(res, axis=2)

    def forward(self, X):
        res = []
        for filter in self.filters:
            res.append(self.__convolution_3d(X, filter))
        res = np.transpose(res, axes=[1, 2, 0])
        return self.activation_function(res)

In [22]:
class MaxPoolLayer(Layer):
    def __init__(self, kernel_size=2):
        self.kernel_size = kernel_size

    def forward(self, X):
        res = np.zeros((X.shape[0] // self.kernel_size,
                        X.shape[1] // self.kernel_size,
                        X.shape[2]))

        for channel in range(X.shape[2]):
            for y in range(0, X.shape[0] - self.kernel_size + 1, self.kernel_size):
                for x in range(0, X.shape[1] - self.kernel_size + 1, self.kernel_size):
                    res [x // self.kernel_size] [y // self.kernel_size] [channel] = \
                        np.max( X[y:(y + self.kernel_size), x:(x + self.kernel_size), channel] )
        return res

In [23]:
class Model_2(Model):
    def __init__(self):
        layer_1 = ConvolutionLayer(3, 8)
        layer_2 = MaxPoolLayer()
        layer_3 = ConvolutionLayer(8, 16)
        layer_4 = MaxPoolLayer()
        self.layers = [layer_1, layer_2, layer_3, layer_4]

In [24]:
np.random.seed(19193)
X = np.random.randint(0, 255, size=(19, 19, 3))
model_2 = Model_2()
model_2(X).shape

(4, 4, 16)

3. Объединить сети из п.2 и п.1. На вход изображение размера 19х19, на выходе вектор из 4 элементов

In [25]:
class Model_3(Model):
    def __call__(self, data):
        model_1 = Model_1()
        model_2 = Model_2()
        return model_1(np.ravel(model_2(data)))

In [26]:
np.random.seed(19193)
X = np.random.randint(0, 255, size=(19, 19, 3))
model_3 = Model_3()
model_3(X)

array([0.4071057 , 0.02058366, 0.04061634, 0.5316943 ])